In [1]:
from doc2vec import Doc2Vec
import numpy as np

### Load Reuters data

In [2]:
word2id = np.load('data/reuters/reuters_word2id.npy').item(0)
id2word = np.load('data/reuters/reuters_id2word.npy').item(0)
tokenized = np.load('data/reuters/reuters_tokenized.npy')
docs = [[id2word[word_id] for word_id in doc] for doc in tokenized]

### Init Doc2Vec model

In [ ]:
doc2vec = Doc2Vec(vocabulary_size=60000, document_size=21000)

### Fit documents to model

In [ ]:
doc2vec.fit(docs)

Building dictionary
vocab size:  59576
doc size:    20578
Initialized
Average loss at step 0: 307.113861
Average loss at step 2000: 268.504996
Average loss at step 4000: 146.992388
Average loss at step 6000: 107.605375
Average loss at step 8000: 88.790577
Average loss at step 10000: 73.688384
Average loss at step 12000: 65.588068
Average loss at step 14000: 74.132314
Average loss at step 16000: 51.539436
Average loss at step 18000: 49.586868
Average loss at step 20000: 43.187461
Average loss at step 22000: 36.416340
Average loss at step 24000: 31.834820
Average loss at step 26000: 29.502952
Average loss at step 28000: 27.483882
Average loss at step 30000: 27.775099
Average loss at step 32000: 31.767761
Average loss at step 34000: 24.695657
Average loss at step 36000: 25.498940
Average loss at step 38000: 19.577735
Average loss at step 40000: 16.043557
Average loss at step 42000: 14.603270
Average loss at step 44000: 14.145684
Average loss at step 46000: 13.665582
Average loss at step 4